#  단어의 맥락을 찾는 N-gram
이전까지 우리가 수행한 단어 빈도 분석은 Uni-gram(유니그램) 방식으로, 텍스트를 단어 1개 단위로 잘라 분석하는 것이다. <br>
이 방식은 어떤 단어가 많이 등장하는지 파악하는 데 유용하지만, 다음과 같은 명확한 한계가 존재한다.

- 맥락 정보의 손실 <br>
단어의 순서와 관계를 무시하므로 문맥이 사라진다.예를 들어, '금리'라는 키워드가 '인상'과 함께 등장했는지, '인하'와 함께 등장했는지 구분할 수 없다. 이는 정반대의 의미임에도 불구하고 Uni-gram 분석에서는 동일하게 '금리'라는 단어의 빈도만 증가시킬 뿐이다.

- 고유 명사 및 복합어 문제 <br>
'삼성전자'나 '기업은행'과 같은 복합 명사는 하나의 개체로 인식되어야 의미가 있다. 하지만 Uni-gram 방식에서는 '삼성'과 '전자', '기업'과 '은행'으로 각각 분리되어 원래의 의미가 훼손된다.

## N-gram의 개념과 종류
N-gram은 Uni-gram의 한계를 극복하기 위해 등장한 개념으로, 텍스트에서 연속적으로 나타나는 N개의 단어 시퀀스를 하나의 단위로 보는 분석 방법이다. 이를 통해 단어의 순서와 주변 단어와의 관계, 즉 문맥 정보를 보존할 수 있다.

### 주요 N-gram 종류
예시 문장: "경제 성장은 둔화되었다"

- Uni-gram (1-gram): 연속된 1개의 단어
  - ['경제', '성장은', '둔화되었다']

- Bi-gram (2-gram): 연속된 2개의 단어 묶음
  - ['경제 성장은', '성장은 둔화되었다']

- Tri-gram (3-gram): 연속된 3개의 단어 묶음
  - ['경제 성장은 둔화되었다']

### 3. N-gram 분석의 활용 및 기대효과
N-gram을 활용하면 단어 간의 관계를 파악하여 더 깊이 있는 분석이 가능하다.

- 주요 구문 및 연어(Collocation) 추출 <br>
 '경제 성장', '금융 위기', '빅테크 기업'처럼 함께 자주 등장하며 특정 의미를 형성하는 구문을 발견할 수 있다. <br>
 이는 단순한 단어 목록보다 훨씬 구체적인 트렌드를 보여준다.

- 분석 정확도 향상 <br>
  '미국 증시'를 하나의 단위로 분석함으로써, '미국'과 '증시'가 별개로 언급된 수많은 문서와 명확히 구분하여 분석의 정확도를 높일 수 있다.

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 38.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# --- 1. 실습용 데이터 ---
documents = [
    "정부, 금리 인상 결정에 증시 하락",
    "미국 증시, 금리 인상 여파로 혼조세",
    "삼성전자, 반도체 실적 발표하며 주가 상승",
    "하반기 반도체 시장 전망과 삼성전자 실적",
    "정부의 부동산 정책 발표와 금리 인하 가능성",
    "미국 기준 금리 동결 소식"
]
df = pd.DataFrame(documents, columns=['제목'])


# --- 2. 텍스트 전처리 (명사 추출) ---
okt = Okt()
def preprocess(text):
    nouns = okt.nouns(text)
    # Bi-gram 분석에서는 한 글자 명사도 의미가 있을 수 있으므로 그대로 사용한다.
    return ' '.join(nouns) # CountVectorizer 입력을 위해 다시 문자열로 합친다.

df['전처리 결과'] = df['제목'].apply(preprocess)


# --- 3. Bi-gram 추출 및 빈도 계산 ---
# ngram_range=(2, 2) 옵션은 텍스트를 2개의 단어 묶음(Bi-gram)으로 자르라는 의미이다.
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(df['전처리 결과'])

# 각 Bi-gram의 빈도수를 합산.
word_counts = X.sum(axis=0)

# 단어(Bi-gram)와 빈도수를 매핑하여 DataFrame으로 만든다.
words = vectorizer.get_feature_names_out()
frequency_df = pd.DataFrame(word_counts.T, index=words, columns=['빈도수'])

# --- 4. 결과 확인 ---
# 빈도수가 높은 순으로 정렬하여 출력합니다.
print("--- Bi-gram 추출 및 빈도 분석 결과 (상위 5개) ---")
display(frequency_df.sort_values(by='빈도수', ascending=False).head(5))

--- Bi-gram 추출 및 빈도 분석 결과 (상위 5개) ---


,빈도수
금리 인상,2
삼성 전자,2
금리 동결,1
금리 인하,1
기준 금리,1
